In [3]:
!pip install jupyter-dash dash pandas plotly


In [5]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the SpaceX data from the provided URL into a pandas DataFrame
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)

# Get max and min payload values
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a Dash application
app = JupyterDash(__name__)

# Prepare launch site options
launch_sites_options = [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()]
launch_sites_options.insert(0, {'label': 'All', 'value': 'All'}) 

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    # Dropdown for Launch Site selection
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_sites_options,
        value='All',  # Default value
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    
    # Pie chart to show the total successful launches count
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    # Slider to select payload range
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: str(i) for i in range(0, 10001, 1000)},
        value=[min_payload, max_payload]
    ),
    
    # Scatter chart to show correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])

# Callback for pie chart
@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value')
)
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'All':
        pie_chart_data = filtered_df['Class'].value_counts().reset_index()
        pie_chart_data.columns = ['Outcome', 'Count']
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        pie_chart_data = filtered_df['Class'].value_counts().reset_index()
        pie_chart_data.columns = ['Outcome', 'Count']
    
    fig = px.pie(
        pie_chart_data,
        values='Count',
        names='Outcome',
        title='Launch Outcomes'
    )
    
    return fig

# Callback for scatter plot
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')]
)
def get_success_payload_scatter(entered_site, payload_range):
    # Initialize the filtered DataFrame
    filtered_df = spacex_df
    
    # Filter based on the selected site
    if entered_site != 'All':
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]

    # Filter by payload range
    filtered_df = filtered_df[
        (filtered_df['Payload Mass (kg)'] >= payload_range[0]) &
        (filtered_df['Payload Mass (kg)'] <= payload_range[1])
    ]

    # Create the scatter plot
    fig = px.scatter(
        filtered_df,
        x='Payload Mass (kg)',  # X-axis: Payload Mass
        y='Class',              # Y-axis: Class (Success/Failure)
        color='Booster Version Category',  # Color by Booster Version
        title='Payload Mass vs. Launch Success',
        labels={'Class': 'Launch Outcome'},
        hover_data=['Launch Site']  # Additional hover information
    )

    return fig

# Run the app in Jupyter Lab
app.run_server(mode='inline', debug=True)


/var/folders/l3/5krkptx11bq7nrcbhgm7snzw0000gn/T/ipykernel_45228/4190297452.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/var/folders/l3/5krkptx11bq7nrcbhgm7snzw0000gn/T/ipykernel_45228/4190297452.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/opt/anaconda3/lib/python3.11/site-packages/dash/dash.py:585: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.

